# STIX spectroscopy example
## Candidate flare: 2022 April 20, M1

In [ ]:
import matplotlib.pyplot as plt
%matplotlib qt
import wget
import os

from sunxspex.sunxspex_fitting import fitter
from sunxspex.sunxspex_fitting import instruments as instr

## Here's an example fit
![fit](example-fit.png)

## Download files

In [ ]:
# Download the even fits file
ident = '2204198941'
url = f'https://datacenter.stix.i4ds.net/download/fits/bsd/{ident}'
fn = wget.download(url)

os.rename(fn, ident + '.fits')
dat_fn = ident + '.fits'

In [ ]:
# Download a corresponding SRM (precalculated using OSPEX)
url = 'https://drive.google.com/uc?export=download&id=1OJBld0uzNSBdtK5P1vhA2nccGfNtXAc_'
srm_fn = wget.download(url)

## Load in the data files, select SRM type and time base (UTC or spacecraft)

In [ ]:
stix_load = instr.StixLoader(
    data_file=dat_fn,
    srm_file=srm_fn,
    srm_type='unattenuated',
    time_base='utc'
)

## Pick fitting ranges and check that they make sense
### Also: add on systematic error _after_ picking fitting ranges.

In [ ]:
stix_load.start_event_time = '2022-04-20T01:07:00'
stix_load.end_event_time = '2022-04-20T01:13:00'


stix_load.start_background_time = '2022-04-19T22:57:00'
stix_load.end_background_time = '2022-04-19T23:12:00'

# must occur after setting times otherwise results are bogus
stix_load.add_systematic_error(0.05)

## Give the data to Sunxspex and fit it
### In this case we can just give decent starting guesses and end up with a fit that works. We don't have to fit thermal/nonthermal separately.

In [ ]:
ss = fitter.SunXspex(stix_load)

In [ ]:
ss.loglikelihood = 'chi2'
ss.model = 'f_vth + thick_fn'

In [ ]:
tp = 'T1_spectrum1'
emp = 'EM1_spectrum1'
thermal_params = [tp, emp]

efp = 'total_eflux1_spectrum1'
ip = 'index1_spectrum1'
ecp = 'e_c1_spectrum1'
nonthermal_params = [efp, ip, ecp]

ss.params[tp] = [20, (5, 50)]
ss.params[emp] = [1000, (0, 1e5)]

ss.params[efp] = [10, (0, 100)]
ss.params[ip] = [6, (3, 10)]
ss.params[ecp] = [10, (5, 20)]

In [ ]:
for p in (thermal_params + nonthermal_params):
    ss.params[p] = 'free'

In [ ]:
ss.energy_fitting_range = [0, 60]
ss.fit()

In [ ]:
axs, *_ = ss.plot()
axs[0].set(xscale='log')
plt.show()

In [ ]:
ss.run_mcmc(walker_spread='mag_order')

In [ ]:
plt.figure()
ss.burn_mcmc = 130
ss.plot_log_prob_chain()

In [ ]:
plt.figure()
ss.corner_mcmc()

In [ ]:
plt.figure(layout='tight', figsize=(10, 6))
axs, *_ = ss.plot()
axs[0].set(xscale='log', title='Example fit')
plt.show()